In [1]:
import pandas as pd
macro_df = pd.read_csv("../data/processed/macro_data_1962on (cleaned).csv")
macro_df.head()

,date,market,yield_curve,oil ($/bbl),copper ($/metric ton),monetary_policy,volatility,stock_bond_corr
0,1/31/1962,68.839996,-2.320,1.52,635.37,2.73,0.093924,NaN
1,2/28/1962,69.959999,-2.310,1.52,647.94,2.71,0.064570,NaN
2,3/31/1962,69.550003,-2.364,1.52,647.50,2.75,0.057723,NaN
3,4/30/1962,65.239998,-2.354,1.52,645.95,2.74,0.098421,NaN
4,5/31/1962,59.630001,-2.310,1.52,645.73,2.70,0.347108,NaN


In [2]:
import numpy as np
macro_df_new = pd.DataFrame()
macro_df_new['date'] = list(macro_df['date'])
macro_df_new['market_log_diff'] = np.log(macro_df['market']).diff()
macro_df_new['yield_curve'] = list(macro_df['yield_curve'])
macro_df_new['yield_curve_diff'] = macro_df['yield_curve'].diff()
macro_df_new['oil_log_diff'] = np.log(macro_df['oil ($/bbl)']).diff()
macro_df_new['copper_log_diff'] = np.log(macro_df['copper ($/metric ton)']).diff()
macro_df_new['monetary_policy'] = list(macro_df['monetary_policy'])
macro_df_new['monetary_policy_diff'] = macro_df['monetary_policy'].diff()
macro_df_new['monetary_policy'] = list(macro_df['monetary_policy'])
macro_df_new['monetary_policy_diff'] = macro_df['monetary_policy'].diff()
macro_df_new['volatility'] = list(macro_df['volatility'])
macro_df_new['volatility_diff'] = macro_df['volatility'].diff()
macro_df_new['stock_bond_corr'] = list(macro_df['stock_bond_corr'])
macro_df_new.head()

,date,market_log_diff,yield_curve,yield_curve_diff,oil_log_diff,copper_log_diff,monetary_policy,monetary_policy_diff,volatility,volatility_diff,stock_bond_corr
0,1/31/1962,NaN,-2.320,NaN,NaN,NaN,2.73,NaN,0.093924,NaN,NaN
1,2/28/1962,0.016139,-2.310,0.010,0.0,0.019591,2.71,-0.02,0.064570,-0.029354,NaN
2,3/31/1962,-0.005878,-2.364,-0.054,0.0,-0.000679,2.75,0.04,0.057723,-0.006847,NaN
3,4/30/1962,-0.063973,-2.354,0.010,0.0,-0.002397,2.74,-0.01,0.098421,0.040698,NaN
4,5/31/1962,-0.089914,-2.310,0.044,0.0,-0.000341,2.70,-0.04,0.347108,0.248687,NaN


In [3]:
macro_df_new = macro_df_new[1:]
macro_df_new.set_index("date", inplace = True)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
macro_std_df = pd.DataFrame(
    scaler.fit_transform(macro_df_new),
    index=macro_df_new.index,
    columns=macro_df_new.columns
)

Euclidean Distance KNN Methods

In [4]:
window = 5
macro_regime = pd.concat([
    macro_std_df.rolling(5).mean(),
    macro_std_df.rolling(5).std()
], axis=1).dropna()

In [5]:
from sklearn.metrics.pairwise import euclidean_distances

dist_matrix = euclidean_distances(macro_regime.values)
dist_df = pd.DataFrame(
    dist_matrix,
    index=macro_regime.index,
    columns=macro_regime.index
)

In [6]:
dist_df

date,5/31/1963,6/30/1963,7/31/1963,8/31/1963,9/30/1963,10/31/1963,11/30/1963,12/31/1963,1/31/1964,2/29/1964,...,1/31/2025,2/28/2025,3/31/2025,4/30/2025,5/31/2025,6/30/2025,7/31/2025,8/31/2025,9/30/2025,10/31/2025
date,,,,,,,,,,,,,,,,,,,,,
5/31/1963,0.000000,0.402112,0.490824,0.533730,0.673183,0.630961,0.611565,0.502374,0.604169,0.880472,...,2.807125,2.774113,2.675997,2.537596,2.552458,2.561130,2.412437,2.233314,2.067210,2.100513
6/30/1963,0.402112,0.000000,0.374247,0.449771,0.457495,0.449355,0.477021,0.404783,0.482615,0.805902,...,2.852566,2.801431,2.671487,2.487273,2.523680,2.536055,2.405044,2.286798,2.165343,2.151898
7/31/1963,0.490824,0.374247,0.000000,0.160451,0.265333,0.234147,0.248960,0.336272,0.470964,0.722098,...,2.891675,2.854089,2.743024,2.553416,2.587717,2.594218,2.451295,2.273826,2.131048,2.141601
8/31/1963,0.533730,0.449771,0.160451,0.000000,0.286452,0.212007,0.226386,0.321214,0.494240,0.699634,...,2.939101,2.893948,2.779413,2.604395,2.608110,2.602095,2.447197,2.271249,2.128722,2.146268
9/30/1963,0.673183,0.457495,0.265333,0.286452,0.000000,0.108485,0.172366,0.381947,0.462730,0.701534,...,2.937865,2.881230,2.752131,2.534535,2.591637,2.597926,2.458494,2.305433,2.179449,2.163410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6/30/2025,2.561130,2.536055,2.594218,2.602095,2.597926,2.581557,2.575793,2.542977,2.501620,2.527906,...,1.401123,1.200393,0.946596,1.230163,0.291021,0.000000,0.402597,1.079124,1.518889,1.328489
7/31/2025,2.412437,2.405044,2.451295,2.447197,2.458494,2.436081,2.427973,2.384792,2.352562,2.365133,...,1.493772,1.333624,1.105110,1.400100,0.603104,0.402597,0.000000,0.770425,1.248667,1.158426
8/31/2025,2.233314,2.286798,2.273826,2.271249,2.305433,2.281446,2.259684,2.197023,2.153620,2.140189,...,1.579218,1.568874,1.479458,1.704397,1.190051,1.079124,0.770425,0.000000,0.644320,0.844567


In [7]:
def nearest_regimes(date, dist_df, k=5):
    return (
        dist_df.loc[date]
        .drop(date)
        .sort_values()
        .head(k)
    )

nearest = nearest_regimes(macro_regime.index[-1], dist_df, k=5)

In [8]:
nearest

date
9/30/2025     0.795915
8/31/2025     0.844567
11/30/2006    0.914343
11/30/1995    1.095636
5/31/1996     1.099625
Name: 10/31/2025, dtype: float64

Mahalanobis Distance KNN Methods

In [9]:
macro_regime = pd.concat([
    macro_std_df.rolling(window).mean(),
    macro_std_df.rolling(window).std()
], axis=1).dropna()
import numpy as np

cov = np.cov(macro_regime.values, rowvar=False)
cov_inv = np.linalg.inv(cov)

In [10]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(
    n_neighbors=5,
    metric="mahalanobis",
    metric_params={"VI": cov_inv}
)

knn.fit(macro_regime.values)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",5
,"radius radius: float, default=1.0Range of parameter space to use by default for :meth:`radius_neighbors`queries.",1.0
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'mahalanobis'
,"p p: float (positive), default=2Parameter for the Minkowski metric fromsklearn.metrics.pairwise.pairwise_distances. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",{'VI': array([[ 9.02...5242734e+01]])}
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None


In [11]:
distances, indices = knn.kneighbors(
    macro_regime.values[-1].reshape(1, -1)
)

nearest_dates = macro_regime.index[indices[0]]
print(nearest_dates)


Index(['10/31/2025', '10/31/1995', '6/30/1998', '11/30/1995', '9/30/1995'], dtype='object', name='date')


Correlation Distance KNN Methods

In [12]:
macro_regime = pd.concat([
    macro_std_df.rolling(window).mean(),
    macro_std_df.rolling(window).std()
], axis=1).dropna()
from sklearn.neighbors import NearestNeighbors

knn_corr = NearestNeighbors(
    n_neighbors=5,
    metric="correlation"
)

knn_corr.fit(macro_regime.values)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",5
,"radius radius: float, default=1.0Range of parameter space to use by default for :meth:`radius_neighbors`queries.",1.0
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'correlation'
,"p p: float (positive), default=2Parameter for the Minkowski metric fromsklearn.metrics.pairwise.pairwise_distances. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None


In [13]:
distances, indices = knn_corr.kneighbors(
    macro_regime.values[-1].reshape(1, -1)
)

nearest_dates = macro_regime.index[indices[0]]

print(nearest_dates)

Index(['10/31/2025', '8/31/2025', '9/30/2025', '2/28/1997', '5/31/1997'], dtype='object', name='date')


Hidden Markov Model Methods

In [14]:
from hmmlearn.hmm import GaussianHMM
macro_regime = pd.concat([
    macro_std_df.rolling(window).mean(),
    macro_std_df.rolling(window).std()
], axis=1).dropna()
from hmmlearn.hmm import GaussianHMM

n_states = 3

hmm = GaussianHMM(
    n_components=n_states,
    covariance_type="full",
    n_iter=1000,
    random_state=42
)

hmm.fit(macro_regime.values)

,n_components,3
,covariance_type,'full'
,min_covar,0.001
,startprob_prior,1.0
,transmat_prior,1.0
,means_prior,0
,means_weight,0
,covars_prior,0.01
,covars_weight,1
,algorithm,'viterbi'
,random_state,42


In [15]:

def nearest_regimes_by_hmm_no_gap(regime_df, date, k=5, exclude_self=True):
    prob_cols = [c for c in regime_df.columns if c.startswith("regime_")]

    p0 = regime_df.loc[date, prob_cols].astype(float).values  

    P = regime_df[prob_cols].astype(float).values           
    d = np.linalg.norm(P - p0, axis=1)                       
    dist = pd.Series(d, index=regime_df.index)

    if exclude_self:
        dist = dist.drop(date)

    return dist.sort_values().head(k)

In [16]:
regime_probs = hmm.predict_proba(macro_regime.values)

regime_df = pd.DataFrame(
    regime_probs,
    index=macro_regime.index,
    columns=[f"regime_{i}" for i in range(n_states)]
)

regime_df["regime"] = hmm.predict(macro_regime.values)

In [17]:
current_date = regime_df.index[-1]
nearest = nearest_regimes_by_hmm_no_gap(regime_df, current_date, k=5)
print(nearest)

date
5/31/2010     1.126637e-47
10/31/2004    1.627887e-47
8/31/2025     2.412896e-47
12/31/2004    2.432915e-47
3/31/2000     2.453510e-47
dtype: float64
